In [ ]:
from glob import glob
import cv2
import numpy as np
from keras.applications.densenet import DenseNet169
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, train_test_split
import tensorflow as tf
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
images_path = {}
images_path["Covid"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.png")
images_path["Covid"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.jpeg")
images_path["Covid"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.jpg")

images_path["NoFindings"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.png")
images_path["NoFindings"] += glob("/content/drive/MyDrive/Source/X-Ray Image DataSet/No_findings/*.jpeg")
images_path["NoFindings"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.jpg")

print(images_path["Covid"])
print(images_path["NoFindings"])

images_class = {
    "Covid": 0,
    "NoFindings": 1
}

['/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/1-s2.0-S1684118220300682-main.pdf-003-b2.png', '/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-000-fig1b.png', '/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-002-fig3b.png', '/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-000-fig1a.png', '/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/41591_2020_819_Fig1_HTML.webp-day10.png', '/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/41591_2020_819_Fig1_HTML.webp-day5.png', '/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/ciaa199.pdf-001-b.png', '/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-001-fig2a.png', '/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-001-fig2b.png', '/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/

In [ ]:
X = []
Y = []

for label in images_path:
    for image_path in images_path[label]:
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224, 224))
        X.append(image)
        Y.append(images_class[label])

In [ ]:
X = np.array(X)
Y = np.array(Y)


In [ ]:
x.shape

(625, 224, 224, 3)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, y):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train=y[train_index]
    y_test=y[test_index]

In [ ]:
X_train=np.array(X_train).reshape(-1,224,224,3).astype('float32')
y_train=np.array(y_train)
X_test=np.array(X_test).reshape(-1,224,224,3).astype('float32')
y_test=np.array(y_test)

In [ ]:
X_train.shape

(500, 224, 224, 3)

In [ ]:
X_train=keras.applications.vgg16.preprocess_input(X_train)
X_test=keras.applications.vgg16.preprocess_input(X_test)

In [ ]:
vgg=keras.applications.vgg16.VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
my_smaller_model =keras.models.Model(vgg.input,vgg.get_layer("block3_conv3").output)
my_smaller_model.trainable = False

In [ ]:
for layer in my_smaller_model.layers:
     layer.trainable = False

In [ ]:
base_model_output = my_smaller_model.output
gap = keras.layers.GlobalAveragePooling2D()(base_model_output)
fc = keras.layers.Dense(units=256, activation="relu")(gap)
output = keras.layers.Dense(1, activation="sigmoid")(fc)

In [ ]:
transfer_model = keras.models.Model(my_smaller_model.input, output)

In [ ]:
transfer_model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
transfer_model.fit(X_train,y_train,epochs=10)

Epoch 1/10
16/16 [==============================] - 17s 200ms/step - loss: 32.1889 - accuracy: 0.7240
Epoch 2/10
16/16 [==============================] - 2s 108ms/step - loss: 2.8190 - accuracy: 0.9220
Epoch 3/10
16/16 [==============================] - 2s 107ms/step - loss: 1.4386 - accuracy: 0.9480
Epoch 4/10
16/16 [==============================] - 2s 108ms/step - loss: 0.8340 - accuracy: 0.9640
Epoch 5/10
16/16 [==============================] - 2s 107ms/step - loss: 0.4891 - accuracy: 0.9680
Epoch 6/10
16/16 [==============================] - 2s 108ms/step - loss: 0.8313 - accuracy: 0.9580
Epoch 7/10
16/16 [==============================] - 2s 108ms/step - loss: 0.5144 - accuracy: 0.9680
Epoch 8/10
16/16 [==============================] - 2s 108ms/step - loss: 0.1758 - accuracy: 0.9840
Epoch 9/10
16/16 [==============================] - 2s 108ms/step - loss: 0.0875 - accuracy: 0.9960
Epoch 10/10
16/16 [==============================] - 2s 108ms/step - loss: 0.4284 - accuracy: 0.97

In [ ]:
my_smaller_model.trainable = True
for layer in my_smaller_model.layers:
     layer.trainable = True 

In [ ]:
ADAM= keras.optimizers.Adam(learning_rate=0.00001)
transfer_model.compile(loss="binary_crossentropy",optimizer=ADAM,metrics=['accuracy'])

In [ ]:
transfer_model.fit(X_train,y_train,batch_size=10,epochs=20)

Epoch 1/20
50/50 [==============================] - 9s 102ms/step - loss: 0.3295 - accuracy: 0.9800
Epoch 2/20
50/50 [==============================] - 5s 102ms/step - loss: 0.7441 - accuracy: 0.9580
Epoch 3/20
50/50 [==============================] - 5s 102ms/step - loss: 0.5602 - accuracy: 0.9680
Epoch 4/20
50/50 [==============================] - 5s 102ms/step - loss: 0.0856 - accuracy: 0.9940
Epoch 5/20
50/50 [==============================] - 5s 103ms/step - loss: 0.4320 - accuracy: 0.9660
Epoch 6/20
50/50 [==============================] - 5s 103ms/step - loss: 0.1637 - accuracy: 0.9860
Epoch 7/20
50/50 [==============================] - 5s 103ms/step - loss: 0.0356 - accuracy: 0.9940
Epoch 8/20
50/50 [==============================] - 5s 104ms/step - loss: 0.0367 - accuracy: 0.9920
Epoch 9/20
50/50 [==============================] - 5s 104ms/step - loss: 0.0471 - accuracy: 0.9880
Epoch 10/20
50/50 [==============================] - 5s 104ms/step - loss: 0.0457 - accuracy: 0.9960

In [ ]:
mse_test = transfer_model.evaluate(X_test, y_test)

4/4 [==============================] - 3s 742ms/step - loss: 0.7594 - accuracy: 0.9760
